In [ ]:
! pip install annoy

# Creating the index of the dataset

In [ ]:
import os,torch
import torchvision
from torchvision import models, transforms
from PIL import Image
from annoy import AnnoyIndex

In [ ]:
image_folder= r'/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog'
images= os.listdir(image_folder)

In [ ]:
model= models.resnet18(pretrained=True)
model.fc= torch.nn.Identity()
model.eval()

In [ ]:
transform= transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [ ]:
annoy_index = AnnoyIndex(512,'angular') #since 512 features are the outputr of the model

In [ ]:
try:
    for i in range(len(images)):
        image= Image.open(os.path.join(image_folder,images[i]))
        input_tensor= transform(image).unsqueeze(0)
    #     print(input_tensor.size())

        if input_tensor.size()[1] == 3:
            output_tensor= model(input_tensor)
    #         print(output_tensor.size())
            annoy_index.add_item(i,output_tensor[0])
except:
    print("skipping corrupt image founnd")
    pass

annoy_index.build(10)
annoy_index.save('dog_index.ann')
print("Indexing the dataset completed !!!")
        

# Inferencing

In [ ]:
#loading the saved annoy index
annoy_index = AnnoyIndex(512,'angular')
annoy_index.load(r'/kaggle/working/dog_index.ann')

In [ ]:
for i in range(len(images)):
        image= Image.open(os.path.join(image_folder,images[i]))
        input_tensor= transform(image).unsqueeze(0)
        if input_tensor.size()[1] == 3:
            output_tensor= model(input_tensor)
            nns= annoy_index.get_nns_by_vector(output_tensor[0],24)
            print('--> ',nns)
            break

In [ ]:
images[628]